In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import regularizers
from keras.layers.recurrent import SimpleRNN
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime
import re

%matplotlib inline
plt.style.use('ggplot')

class TechnicalTerm():
    @classmethod
    def bolinger_band(self, raw_data):
        bolinger_option = ["bolinger_upper1", "bolinger_lower1", "bolinger_upper2", "bolinger_lower2"]
        base = raw_data[["Close"]].rolling(window=25).mean()
        std = raw_data[["Close"]].rolling(window=25).std()

        for opt in bolinger_option:
            if opt == "bolinger_upper1":
                raw_data[opt] = base + std
            elif opt == "bolinger_lower1":
                raw_data[opt] = base - std
            elif opt == "bolinger_upper2":
                raw_data[opt] = base + 2 * std
            elif opt == "bolinger_lower2":
                raw_data[opt] = base - 2 * std

        data = raw_data.dropna()
        return data

    @classmethod
    def conversion(self, raw_data):
        raw_data["rol_high"] = raw_data[["High"]].rolling(window=9*60*24).max()
        raw_data["rol_low"] = raw_data[["Low"]].rolling(window=9*60*24).min()
        raw_data = raw_data.dropna()

        high = raw_data[["rol_high"]].values
        low = raw_data[["rol_low"]].values
        raw_data["conversion"] = np.reshape((high + low) / 2, (-1, ))
        data = raw_data

        return data
    
def create_data(data, label_data, term):
    created_data = []
    nested_data = []
    label = []

    label_data= np.reshape(label_data, (-1, ))
    for index, dt in enumerate(data):
        nested_data.append(dt)

        if len(nested_data) == term:
            created_data.append(nested_data)
            label.append(label_data[index])
            nested_data = []

    label = np.reshape(np.array(label), (-1, ))
    return np.array(created_data), label

def split_data(train, label, testing_rate=0.7):
    train_x, test_x = train[1:int(len(train) * testing_rate)], train[1 + int(len(train) * testing_rate):len(train)]
    train_y, test_y = label[1:int(len(label) * testing_rate)], label[1 + int(len(label) * testing_rate):len(label)]
    return train_x, train_y, test_x, test_y

def training(x_train, y_train, x_test, y_test, term, option_length, neurons=128, dropout=0.25, epoch=20):
    model = Sequential()
    model.add(SimpleRNN(neurons, batch_input_shape=(None, term, option_length),
                   recurrent_regularizer=regularizers.l2(0.), activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(1, activation='linear'))

    model.compile(loss="mean_squared_error", optimizer="adam")

    # -------------training-------------
    output = model.fit(x_train, y_train, epochs=epoch, verbose=1)

    predicted_price = model.predict(x_test)
    datas = pd.DataFrame(
        data = {
            'real_price': np.reshape(y_test, (-1, )),
            'predicted_price': np.reshape(predicted_price, (-1, ))
        }
    )
    
    plt.plot(output.history['loss'])
    plt.title('model loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['loss'], loc='lower right')
    
    datas.plot(figsize=(16, 10))
    plt.show()
    print(mean_absolute_error(np.reshape(y_test, (-1, )),
                             np.reshape(predicted_price, (-1, ))))

raw_data = pd.read_csv("/Users/shimizurei/CryptoCurrencyPredictor/src/coin.csv").dropna()

# append hour
hour = []
hour_data = np.reshape(raw_data[["Timestamp"]].values, (-1, ))
for timestamp in hour_data:
    time = datetime.datetime.fromtimestamp(int(timestamp))
    hour.append(re.match(r"\d{4}-\d{1,2}-\d{1,2} (\d\d):\d\d:\d\d", str(time)).group(1))

raw_data["Hour"] = hour
raw_data = raw_data.drop(["Timestamp"], axis=1)

# append bolinger band
raw_data = TechnicalTerm.bolinger_band(raw_data)
# append conversion line
raw_data = TechnicalTerm.conversion(raw_data)

print(raw_data.head(2))
options = ["Open", "High", "Low", "Close", "Volume_(BTC)", "Volume_(Currency)", "Hour",
           "bolinger_upper1", "bolinger_lower1", "bolinger_upper2", "bolinger_lower2", "conversion"]
df_train = raw_data[options]
df_label = raw_data[["Weighted_Price"]]
term = 10

x_train, y_train, x_test, y_test = split_data(df_train, df_label)

# ----train data noramlization---------
x_train = x_train[options].values
y_train = y_train.values
x_train, y_train = create_data(x_train, y_train, term)
# -------------------------------------

# ----train data noramlization---------
x_test = x_test[options].values
y_test = y_test.values
x_test, y_test = create_data(x_test, y_test, term)
# -------------------------------------

training(x_train, y_train, x_test, y_test, term, len(options), neurons=256, dropout=0.25, epoch=40)


/Users/shimizurei/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


        Open   High    Low  Close  Volume_(BTC)  Volume_(Currency)  \
12983  41210  41210  41210  41210         0.101            4162.21   
12984  41210  41210  41210  41210         0.101            4162.21   

       Weighted_Price Hour  bolinger_upper1  bolinger_lower1  bolinger_upper2  \
12983         41210.0   15     41210.001054     41209.998946     41210.002109   
12984         41210.0   15     41210.001054     41209.998946     41210.002109   

       bolinger_lower2  rol_high  rol_low  conversion  
12983     41209.997891   45000.0  35620.0     40310.0  
12984     41209.997891   45000.0  35620.0     40310.0  


Epoch 1/40
    32/108433 [..............................] - ETA: 275s - loss: 4438655488.0000

   128/108433 [..............................] - ETA: 116s - loss: 9244306432.0000

   224/108433 [..............................] - ETA: 91s - loss: 5563973261.7143 

...

  8096/108433 [=>............................] - ETA: 36s - loss: 96568699.5099

  8224/108433 [=>............................] - ETA: 36s - loss: 96678784.3268

  8352/108433 [=>............................] - ETA: 36s - loss: 96360867.8697

  8480/108433 [=>............................] - ETA: 36s - loss: 96541843.8717

  8608/108433 [=>............................] - ETA: 36s - loss: 96504959.6803